# 03-JSON 结构化输出 - DeepSeek API

In [ ]:
from openai import OpenAI
import json
import os

client = OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY", "your-api-key"),
    base_url="https://api.deepseek.com",
)

## 基础 JSON 输出

In [ ]:
# JSON 输出
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant. Always respond in valid JSON format."
        },
        {
            "role": "user",
            "content": "Extract name and age from: John is 25 years old."
        }
    ],
    response_format={"type": "json_object"},
    temperature=0.1
)

data = json.loads(response.choices[0].message.content)
print(json.dumps(data, indent=2))

## 实体抽取

In [ ]:
def extract_entities(text):
    """从文本中提取实体"""
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {
                "role": "system",
                "content": """Extract entities and return JSON format:
{
    "people": ["name1", "name2"],
    "organizations": ["org1", "org2"],
    "locations": ["loc1", "loc2"],
    "dates": ["date1", "date2"]
}"""
            },
            {
                "role": "user",
                "content": text
            }
        ],
        response_format={"type": "json_object"},
        temperature=0.1
    )
    
    return json.loads(response.choices[0].message.content)

# 使用
text = "Apple Inc. announced on January 10, 2024 that Tim Cook will visit Beijing."
entities = extract_entities(text)
print(json.dumps(entities, indent=2, ensure_ascii=False))

## 思考模型 JSON 输出

In [ ]:
# deepseek-reasoner 也支持 JSON 输出
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=[
        {
            "role": "user",
            "content": """分析以下数学问题，给出解题步骤和答案：
问题：一个水箱有两个进水管 A 和 B...
请以 JSON 格式返回：
{
    "analysis": "问题分析",
    "steps": ["步骤1", "步骤2", ...],
    "answer": "最终答案"
}"""
        }
    ],
    response_format={"type": "json_object"},
    max_tokens=4096
)

result = json.loads(response.choices[0].message.content)
print(f"分析: {result['analysis']}")
print(f"步骤: {result['steps']}")
print(f"答案: {result['answer']}")

# 查看思考过程
reasoning = getattr(response.choices[0].message, "reasoning_content", None)
if reasoning:
    print(f"\n🧠 思考过程: {reasoning[:200]}...")

## 批量数据处理

In [ ]:
def analyze_sentiment(text):
    """情感分析"""
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {
                "role": "system",
                "content": """Analyze sentiment and return JSON:
{
    "sentiment": "positive/negative/neutral",
    "score": 1-10,
    "key_points": ["point1", "point2"]
}"""
            },
            {"role": "user", "content": text}
        ],
        response_format={"type": "json_object"},
        temperature=0.2
    )
    
    return json.loads(response.choices[0].message.content)

# 批量分析
reviews = [
    "Great product! Fast delivery.",
    "Not good, broke after one day.",
    "Average quality, acceptable price."
]

for i, review in enumerate(reviews, 1):
    result = analyze_sentiment(review)
    print(f"\nReview {i}: {review}")
    print(f"  Sentiment: {result['sentiment']} (score: {result['score']})")
    print(f"  Key points: {result['key_points']}")